# **Ansatz Circuit Configuration Testbench - Quantum Machine Learning Capstone 2022**

#### **Capstone Members ⸻** Carson Darling, Brandon Downs, Christopher Haddox, Brightan Hsu, Matthew Jurenka

#### **Sponsor ⸻** Dr. Gennaro De Luca

<br>


## **Introduction**

The purpose of this Jupyter Notebook is to serve as a testbench for the quantum machine learning capstone group. This testbench allows for the testing of a quantum variational classifier with different ansatz configurations on three different datasets. Each dataset consists of instances containing a binary classification over 4 numeric features. The circuits will all exhibit rotational encoding over 4 qubits, allowing a qubit for each feature. The datasets are as follows:

&emsp;&emsp;[Iris Dataset](https://archive.ics.uci.edu/ml/datasets/iris) ⸻ 3 classes of 150 instances of plant measures, where each class refers to a type of iris plant. This dataset will be truncated to only 2 classes.

&emsp;&emsp;[Banknote Dataset](https://archive.ics.uci.edu/ml/datasets/banknote+authentication)⸻ 2 classes consisting of 1372 instances of banknote-like specimen, where each class refers to forgery or authenticate.

&emsp;&emsp;[Transfusion Dataset](https://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center) ⸻ 2 classes consisting of 748 donors from the donor database, where each class refers to donation in March 2007.
<br>

### Packages and Non-Standard Python Package Installation

The non-standard python packages used by this TestBench are TQDM, SKLearn, Pennylane, and Pandas.
Uncomment and execute the method **clean_install()** to execute the installation via PIP. The environment must have Python 3.6+ and PIP installed.

In [1]:
import subprocess
import sys


def pip_install(package):
    subprocess.run([sys.executable, "-m", "pip", "install", package])


def clean_install():
    [pip_install(package)
     for package in ['tqdm', 'sklearn', 'pennylane', 'pandas']]

# clean_install()


In [2]:
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm_notebook
from pennylane import numpy as np
import pennylane as qml
import random as rand
import time as time
import pandas as pd
import os as os


## **Methodology**


### **Independent Variables**


In [3]:
# Relating to the tests
NUM_ITERATIONS = 3

# Relating to Circuit Configurations
DEV = qml.device("default.qubit", wires=4)
NUM_QUBITS = 4

# Relating to the Variational Classifier
NUM_LAYERS = 6
BATCH_SIZE = 5
STEP_SIZE = .1
MAX_EPOCHS = 100
BIAS_INIT = np.array(0.0, requires_grad=True)
OPTIMIZER = qml.optimize.AdamOptimizer(STEP_SIZE)
WEIGHTS_INIT = 0.01 * np.random.randn(NUM_LAYERS, NUM_QUBITS, 3, requires_grad=True)

# Relating to the datasets
paths = ['~/Documents/QML/iris.data', '~/Documents/QML/banknote.data',
         '~/Documents/QML/transfusion.data']
DATAFRAMES = [((os.path.splitext(os.path.basename(path))[0]).capitalize(), pd.read_csv(
    path, names=['a0', 'a1', 'a2', 'aNUM_ITERATIONS', 'target'])) for path in paths]

### **Dependent Variable** - Ansatz Circuit Configuration


This global variable will serve as a function reference to a specific tested ansatz circuit configuration during test iterations. The implementation of function reference serves for increased readability in circuit configurations by encapsulating encoding techniques within a circuit configuration.


In [4]:
CURRENT_TEST_CIRCUIT = None


### **Ansatz Circuit Configuration Library**


In [5]:
# 3 Basic types of Ansatz
def layered_gate_circuit(params, x):
    pass


def alternating_operator_circuit():
    pass


def tensor_network_circuit():
    pass


@qml.qnode(DEV)
def qaoa_circuit(features):
    # TODO for matthew
    # https://discuss.pennylane.ai/t/qaoa-embedding-layer/1724/2
    # https://docs.pennylane.ai/en/latest/code/api/pennylane.QAOAEmbedding.html

    return None


# Pennylane Circuit from Quantum Variational Classifier
@qml.qnode(DEV)
def pennylane_circuit(weights, features):
    rotational_encoding(features)

    for W in weights:
        qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
        qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
        qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
        qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 0])

    return qml.expval(qml.PauliZ(0))

# https://quantaggle.com/algorithms/ansatz/


@qml.qnode(DEV)
def hardware_efficient_circuit(weights, features):
    rotational_encoding(features)

    for W in weights[:-1]:
        qml.RY(W[0, 0], wires=0)
        qml.RZ(W[0, 1], wires=0)
        qml.RY(W[1, 0], wires=0)
        qml.RZ(W[1, 1], wires=0)
        qml.RY(W[2, 0], wires=0)
        qml.RZ(W[2, 1], wires=0)
        qml.RY(W[3, 0], wires=0)
        qml.RZ(W[3, 1], wires=0)
        qml.CZ(wires=[0, 1])
        qml.CZ(wires=[2, 3])
        qml.CZ(wires=[1, 2])

    W = weights[-1]
    qml.RY(W[0, 0], wires=0)
    qml.RZ(W[0, 1], wires=0)
    qml.RY(W[1, 0], wires=0)
    qml.RZ(W[1, 1], wires=0)
    qml.RY(W[2, 0], wires=0)
    qml.RZ(W[2, 1], wires=0)
    qml.RY(W[3, 0], wires=0)
    qml.RZ(W[3, 1], wires=0)

    return qml.expval(qml.PauliZ(0))

# https://pennylane.ai/qml/glossary/circuit_ansatz.html#a-parametrized-b-parametrized


@qml.qnode(DEV)
def pennylane_ab_paramaterized(weights, features):
    rotational_encoding(features)

    for W in weights:
        qml.CRot(W[0, 0], W[0, 1], W[0, 2], wires=[0, 2])
        qml.CRot(W[1, 0], W[1, 1], W[1, 2], wires=[1, 3])
        qml.CRot(W[2, 0], W[2, 1], W[2, 2], wires=[2, 0])
        qml.CRot(W[3, 0], W[3, 1], W[3, 2], wires=[3, 1])

    return qml.expval(qml.PauliZ(0))

# https://arxiv.org/pdf/1612.02806.pdf


@qml.qnode(DEV)
def pairwise_controlled_rot(weights, features):
    rotational_encoding(features)

    for W in weights:
        qml.CRot(W[0, 0], W[0, 1], W[0, 2], wires=[0, 1])
        qml.CRot(W[0, 0], W[0, 1], W[0, 2], wires=[0, 2])
        qml.CRot(W[0, 0], W[0, 1], W[0, 2], wires=[0, 3])

        qml.CRot(W[1, 0], W[1, 1], W[1, 2], wires=[1, 0])
        qml.CRot(W[1, 0], W[1, 1], W[1, 2], wires=[1, 2])
        qml.CRot(W[1, 0], W[1, 1], W[1, 2], wires=[1, 3])

        qml.CRot(W[2, 0], W[2, 1], W[2, 2], wires=[2, 3])
        qml.CRot(W[2, 0], W[2, 1], W[2, 2], wires=[2, 1])
        qml.CRot(W[2, 0], W[2, 1], W[2, 2], wires=[2, 0])

        qml.CRot(W[3, 0], W[3, 1], W[3, 2], wires=[3, 2])
        qml.CRot(W[3, 0], W[3, 1], W[3, 2], wires=[3, 1])
        qml.CRot(W[3, 0], W[3, 1], W[3, 2], wires=[3, 0])

    return qml.expval(qml.PauliZ(0))


circuit_library = [pennylane_circuit, hardware_efficient_circuit,
                   pennylane_ab_paramaterized, pairwise_controlled_rot]


### **Quantum Variational Classifier**


Below is the variational classifier and its supporting functions. This variational classifier model is adapted from the [pennylane variational classifier demo](https://pennylane.ai/qml/demos/tutorial_variational_classifier.html).


In [6]:
def variational_classifier(weights, bias, angles):
    return CURRENT_TEST_CIRCUIT(weights, angles) + bias


def cost(weights, bias, features, labels):
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)


In [7]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss


In [8]:
header_template = '\t{:<7}   {:<7}   {:<16}   {:<15}'
bar_format="{bar}{n_fmt}/{total_fmt} [{desc} {elapsed}, {rate_fmt}]"

def print_header():
    # Print the header for the current Iteration
    print(header_template.replace(':', ':-').format('', '', '', ''))
    print(header_template.format(
        *['Epoch', 'Cost', 'Train_Accuracy', 'Test_Accuracy']))
    print(header_template.replace(':', ':-').format('', '', '', ''))


def print_summary(max_cost, max_train, max_test):
    print(header_template.replace(':', ':-').format('', '', '', ''))
    print(header_template.format(*['Maxima', f'{max_cost:0.3f}',
          f'{max_train:0.7f}', f'{max_test:0.7f}']))
    print(header_template.replace(':', ':-').format('', '', '', ''))


def train_classifier(dataset):

    # Preprocess the data and seperate into train and test sets. Initialize the weights, bias.
    features, labels = preprocess(dataset[1])
    X_train, X_test, y_train, y_test = train_test_split(
        features, labels, test_size=0.25, random_state=rand.randint(0, 100))
    weights, bias = WEIGHTS_INIT, BIAS_INIT
    max_cost = max_train = max_test = 0

    epoch_progress = tqdm_notebook(range(MAX_EPOCHS), desc='Classifier Progress', bar_format=bar_format)

    with epoch_progress as pbar:

        for epoch_index in range(MAX_EPOCHS):

            if not epoch_index:
                print_header()

            # Update the weights by one optimizer step
            batch_index = np.random.randint(0, len(X_train), (BATCH_SIZE,))

            X_train_batch = X_train[batch_index]
            y_train_batch = y_train[batch_index]
            weights, bias, _, _ = OPTIMIZER.step(
                cost, weights, bias, X_train_batch, y_train_batch)

            # Compute predictions on train and validation set
            predictions_train = [np.sign(variational_classifier(
                weights, bias, value)) for value in X_train]
            predictions_test = [np.sign(variational_classifier(
                weights, bias, value)) for value in X_test]

            # Compute accuracy on train and validation set
            accuracy_train = accuracy(y_train, predictions_train)
            accuracy_test = accuracy(y_test, predictions_test)
            epoch_cost = cost(weights, bias, features, labels)

            # Tabulate a summary of the current epoch
            print(header_template.format(*[f'{epoch_index:4d}', f'{epoch_cost:0.3f}',
                f'{accuracy_train:0.7f}', f'{accuracy_test:0.7f}']))
            max_cost = epoch_cost if epoch_cost > max_cost else max_cost
            max_train = accuracy_train if accuracy_test > max_train else max_train
            max_test = accuracy_test if accuracy_test > max_test else max_test

            # Break if train and test validation is 100% accuracy.
            if accuracy_test == accuracy_train == 1:
                pbar.update(MAX_EPOCHS-epoch_index)
                pbar.close()
                break
            else:
                pbar.update(1)




    print_summary(max_cost, max_train, max_test)


### **Data Preparation, Preprocessing, and Encoding**


In [9]:
def preprocess(df):

    df.target = df.target.map(
        {df.target.unique()[0]: -1, df.target.unique()[1]: 1})

    if df.target.value_counts()[-1] >= 100 and df.target.value_counts()[1] >= 100:
        df = pd.concat([
            df[(df.target == -1)].sample(n=100, replace=False,
                                         random_state=rand.randint(0, 100)),
            df[(df.target == 1)].sample(n=100, replace=False,
                                        random_state=rand.randint(0, 100))
        ])
    else:
        df = df[(df.target == -1) | (df.target == 1)]

    X = np.array(df)[:, 0:4]
    features = 2 * np.pi * (X - np.min(X)) / (np.max(X) - np.min(X))
    labels = np.array(df)[:, -1]

    return features, labels


def rotational_encoding(x):
    qml.Rot(x[0], x[0], x[0], wires=0)
    qml.Rot(x[1], x[1], x[1], wires=1)
    qml.Rot(x[2], x[2], x[2], wires=2)
    qml.Rot(x[3], x[3], x[3], wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])
    return np.array(x)


### **Test Execution**


In [10]:
def TEST(circuit):

    # Set the tested circuit global to the circuit
    global CURRENT_TEST_CIRCUIT
    CURRENT_TEST_CIRCUIT = circuit

    for dataset in tqdm_notebook(DATAFRAMES, desc=f'{circuit.__name__.capitalize()} Progress', bar_format=bar_format):

        for iteration in tqdm_notebook(range(NUM_ITERATIONS), desc=f'{dataset[0]} Progress', bar_format=bar_format):

            train_classifier(dataset)



[TEST(circuit) for circuit in circuit_library]

# TODO:
# 1. Collect Data and send to CSV
# 2. Select the best run from optimization steps
#     Maybe Double the step size and run 3 iterations of each step size? Cast out outliers during data processing
#     Maybe Run different optimizers as well?
# 3. Graph it on completion


# Q for group
# 1.  Increase Iterations
# AdamOPtimizer <-
# MomemtumOptimizer <-
# 2. Undersampling
# 3. Iris -> flower1 flower2 flower3 linearity



# TIME THE ITERATIONS???

          0/3 [Pennylane_circuit Progress 00:00, ?it/s]

          0/3 [Iris Progress 00:00, ?it/s]

          0/100 [Classifier Progress 00:00, ?it/s]

	-------   -------   ----------------   ---------------
	Epoch     Cost      Train_Accuracy     Test_Accuracy  
	-------   -------   ----------------   ---------------
	   0      2.122     0.0133333          0.0000000      
	   1      1.817     0.2266667          0.2400000      
	   2      1.247     0.4266667          0.4800000      
	   3      0.725     0.6666667          0.6400000      
	   4      0.372     0.9733333          1.0000000      
	   5      0.280     1.0000000          1.0000000      
	-------   -------   ----------------   ---------------
	Maxima    2.122     1.0000000          1.0000000      
	-------   -------   ----------------   ---------------


          0/100 [Classifier Progress 00:00, ?it/s]

...

          0/100 [Classifier Progress 00:00, ?it/s]

	-------   -------   ----------------   ---------------
	Epoch     Cost      Train_Accuracy     Test_Accuracy  
	-------   -------   ----------------   ---------------
	   0      1.407     0.5600000          0.4800000      
	   1      1.341     0.5600000          0.4400000      
	   2      1.280     0.5600000          0.4400000      
	   3      1.220     0.5466667          0.4400000      
	   4      1.157     0.5466667          0.4400000      
	   5      1.097     0.5466667          0.4400000      
	   6      1.053     0.5800000          0.4200000      
	   7      1.025     0.6133333          0.5000000      
	   8      1.009     0.6133333          0.5000000      
	   9      1.002     0.6133333          0.5600000      
	  10      1.001     0.6133333          0.5600000      
	  11      1.004     0.6333333          0.5400000      
	  12      1.013     0.6466667          0.5600000      
	  13      1.026     0.6466667          0.5600000      
	  14      1.037     0.6400000          0.580000

          0/100 [Classifier Progress 00:00, ?it/s]

	-------   -------   ----------------   ---------------
	Epoch     Cost      Train_Accuracy     Test_Accuracy  
	-------   -------   ----------------   ---------------
	   0      1.482     0.5000000          0.6200000      
	   1      1.461     0.5000000          0.6200000      
	   2      1.418     0.5000000          0.6200000      
	   3      1.382     0.5000000          0.6200000      
	   4      1.329     0.5000000          0.6200000      
	   5      1.277     0.5000000          0.6200000      
	   6      1.224     0.5133333          0.6200000      
	   7      1.175     0.5000000          0.6200000      
	   8      1.135     0.5000000          0.6200000      
	   9      1.109     0.5066667          0.6200000      
	  10      1.090     0.5066667          0.6200000      
	  11      1.078     0.5400000          0.6200000      
	  12      1.057     0.5400000          0.6200000      
	  13      1.027     0.5066667          0.6200000      
	  14      1.005     0.5066667          0.620000

[None, None, None, None]

## **Results**



## **Discussion**

## **Conclusion**
